In [1]:
import torch
print(torch.__version__)
print(torch.cuda.is_available())


2.9.1+cpu
False


## Technical Notes: PyTorch Environment Issue on Windows

During development, importing PyTorch caused the Jupyter kernel to crash on Windows
(e.g., `torch_python.dll` loading errors and WinError 4551). This issue was not related
to model code or the HuggingFace Trainer API, but to Windows application control policies
blocking PyTorch native DLLs in the original Conda environment.

### Solution
To ensure stability and reproducibility, a clean CPU-only PyTorch environment was created:

- A dedicated Conda environment (`llm_cpu`) with Python 3.10
- Installation of the official CPU-only PyTorch wheels via pip
- Explicit registration and selection of the Jupyter kernel for this environment

This approach avoids GPU/CUDA dependencies, bypasses system-level DLL restrictions,
and provides a stable setup for fine-tuning transformer models on CPU.

Baseline experiments in this project were therefore conducted on CPU.


In [2]:
import sys
print(sys.executable)



c:\Users\crellamaybe\anaconda3\envs\llm_cpu\python.exe


In [3]:
import os
import numpy as np
import pandas as pd

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

from transformers import (
    BertTokenizer,
    BertForSequenceClassification,
    TrainingArguments,
    Trainer,
)

#不提前引入 accelerate（Trainer 会处理） 

c:\Users\crellamaybe\anaconda3\envs\llm_cpu\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
df = pd.read_csv("../data/processed/train_processed.csv")
df.head()


,input_text,label
0,Prompt: Is it morally right to try to have a c...,0
1,Prompt: What is the difference between marriag...,1
2,Prompt: explain function calling. how would yo...,2
3,Prompt: How can I create a test set for a very...,0
4,Prompt: What is the best way to travel from Te...,1


In [6]:
df.info()
df["label"].value_counts()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57477 entries, 0 to 57476
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   input_text  57477 non-null  object
 1   label       57477 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 898.2+ KB


label
0    20064
1    19652
2    17761
Name: count, dtype: int64

In [8]:

#Train / Validation Split
train_texts, val_texts, train_labels, val_labels = train_test_split(
    df["input_text"].tolist(),
    df["label"].tolist(),
    test_size=0.2,
    random_state=42,
    stratify=df["label"].tolist(),
)

I used stratified train-validation splitting to preserve the original class distribution, especially because the dataset is imbalanced and the tie class is relatively rare. This ensures that evaluation metrics remain meaningful and comparable across experiments.

Tokenizer

In [9]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

train_encodings = tokenizer(
    train_texts,
    truncation=True,
    padding=True,
    max_length=512,
)

val_encodings = tokenizer(
    val_texts,
    truncation=True,
    padding=True,
    max_length=512,
)


Dataset

In [11]:
class ClassificationDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)


In [12]:
train_dataset = ClassificationDataset(train_encodings, train_labels)
val_dataset = ClassificationDataset(val_encodings, val_labels)


In [13]:
model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    num_labels=3
)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [14]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=1)

    precision, recall, f1, _ = precision_recall_fscore_support(
        labels, preds, average="weighted"
    )
    acc = accuracy_score(labels, preds)

    return {
        "accuracy": acc,
        "precision": precision,
        "recall": recall,
        "f1": f1,
    }


In [ ]:
training_args = TrainingArguments(
    output_dir="./results",            
    per_device_train_batch_size=2,      # 🔥 CPU 必须小
    per_device_eval_batch_size=4,
    eval_steps=500,
    logging_steps=100,
    save_strategy="no",                 # 不存 checkpoint（省时间）
    use_cpu=True,                       # 替代 no_cuda（新 API）
    fp16=False,
)


In [20]:
# ================================
# 🔎 Sanity check: use small subset
# ================================
SANITY_CHECK = True

if SANITY_CHECK:
    train_dataset = torch.utils.data.Subset(train_dataset, range(500))
    val_dataset   = torch.utils.data.Subset(val_dataset, range(100))


In [21]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
)
trainer.train()

Step,Training Loss


KeyboardInterrupt: 

The full training pipeline is implemented, but due to CPU constraints I did not run full fine-tuning locally. In practice, this setup is designed for GPU-based training, while local runs focus on validating correctness and inference.

中文

训练流程是完整实现的，但考虑到 CPU 的限制，我没有在本地完整跑完 BERT 微调。实际使用中这类模型通常在 GPU 或云端训练，本地主要用于验证 pipeline 和推理流程。


## 🧠 What Is the Trainer Training? | Trainer 在训练什么？

### 🇬🇧 English Explanation

In this project, HuggingFace’s `Trainer` is used as a **training controller**, not as a model itself.

The actual model being trained is:

- **BERT encoder (`bert-base-uncased`)**
- **A classification head on top of BERT**, producing 3 logits:
  - Response A wins
  - Response B wins
  - Tie

Formally, the architecture can be written as:

```

Input Text (Prompt + Response A + Response B)
↓
Tokenizer (input_ids, attention_mask)
↓
BERT Encoder (Transformer layers)
↓
[CLS] representation
↓
Linear Classification Head (hidden_size → 3)
↓
Logits: [score_A, score_B, score_Tie]

```

During training, the `Trainer` performs the following steps at each iteration:

1. Loads a batch of tokenized inputs and labels
2. Runs a forward pass through the model
3. Computes the **cross-entropy loss** using the ground-truth label  
4. Backpropagates gradients through both the BERT encoder and the classification head
5. Updates model parameters using gradient descent
6. Periodically evaluates performance on the validation set

The optimization objective is to maximize the probability assigned to the human-preferred response.

This means the model learns **implicit human preference patterns**, such as:
- relevance to the prompt  
- coherence and correctness  
- helpfulness and clarity  
- stylistic appropriateness  

Conceptually, this model acts as a **supervised preference model**, which corresponds to the reward modeling stage commonly used in **RLHF (Reinforcement Learning from Human Feedback)** pipelines.

---

### 🇨🇳 中文解释

在本项目中，HuggingFace 的 `Trainer` 并不是模型本身，而是一个**训练流程控制器**。

真正被训练的模型由两部分组成：

- **BERT 编码器（bert-base-uncased）**
- **BERT 顶部的分类头（3 分类）**，分别表示：
  - 回答 A 更受偏好
  - 回答 B 更受偏好
  - 两者打平（tie）

模型结构可以概括为：

```

输入文本（Prompt + Response A + Response B）
↓
分词器（Tokenizer）
↓
BERT 编码器（多层 Transformer）
↓
[CLS] 向量表示
↓
线性分类层（输出 3 个 logit）

```

在训练过程中，`Trainer` 在每一个 step 中会执行：

1. 读取一个 batch 的输入文本和对应标签  
2. 进行前向传播，计算模型输出  
3. 使用 **交叉熵损失（Cross-Entropy Loss）** 计算预测与真实标签之间的差距  
4. 反向传播梯度，更新 BERT 编码器和分类头的参数  
5. 按设定的步数在验证集上进行评估  

训练的目标是：  
**让模型给“人类更偏好的回答”分配更高的概率。**

因此，模型实际上学习的是一种**人类偏好函数**，例如：
- 回答是否切题  
- 是否逻辑清晰、信息充分  
- 表达是否自然、有帮助  

从方法论上看，这一模型可以视为 **RLHF（基于人类反馈的强化学习）流程中的监督式偏好模型（preference / reward model）阶段**，而不是简单的文本分类任务。

---

### 🔑 Key Takeaway | 核心总结

- The `Trainer` optimizes **both the language representation (BERT) and the decision layer**.
- The task is **preference learning**, not ordinary classification.
- This setup mirrors real-world **human feedback modeling** used in modern LLM alignment pipelines.

```

---

